## Summary

### Feature Engineering Complete! ✅

**Processed Features:**
- Missing values handled (median imputation)
- Derived features created (engagement score, efficiency metrics)
- Features scaled (StandardScaler)
- Top features selected (Mutual Information)

**Datasets Ready:**
- ✅ Delivery Delay Prediction
- ✅ Order Cancellation Prediction
- ✅ Customer Satisfaction Prediction

**Next Steps:**
➡️ **Notebook 03**: Train ML models using the processed datasets

---

## 1. Import Required Libraries

In [39]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path

# Add src directory to path
sys.path.append(str(Path().absolute().parent / 'src'))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import joblib

# Import custom loader
from load_training_data import SnowflakeDataLoader

import warnings
warnings.filterwarnings('ignore')

## 2. Load Data from Snowflake

In [40]:
# Initialize data loader
loader = SnowflakeDataLoader()

# Load OBT data (using sample_size instead of limit)
df = loader.load_obt_data(sample_size=50000)
print(f"Loaded {len(df)} rows")
print(f"Shape: {df.shape}")

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.6.0, Python Version: 3.11.3, Platform: Windows-10-10.0.26100-SP0
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:load_training_data:Connected to Snowflake successfully
INFO:load_training_data:Executing query:

        SELECT * FROM gold_obt_orders_ml_export
        WHERE 1=1
        
  LIMIT 50000
INFO:snowflake.connector.cursor:query: [SELECT * FROM gold_obt_orders_ml_export WHERE 1=1  LIMIT 50000]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:load_training_data:Loaded 50,000 rows with 55 features


Loaded 50000 rows
Shape: (50000, 55)


## 3. Handle Missing Values

In [41]:
# Calculate missing percentage
missing_pct = (df.isnull().sum() / len(df)) * 100
print("Columns with missing values:")
print(missing_pct[missing_pct > 0].sort_values(ascending=False))

# Drop columns with >50% missing values
cols_to_drop = missing_pct[missing_pct > 50].index.tolist()
if cols_to_drop:
    print(f"\nDropping columns: {cols_to_drop}")
    df = df.drop(columns=cols_to_drop)

# Fill remaining missing values with median for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

Columns with missing values:
STDDEV_ITEM_PRICE       89.856
REVIEW_SCORE             0.700
TARGET_REVIEW_SCORE      0.700
ACTUAL_DELIVERY_DAYS     0.008
TARGET_DELIVERY_DAYS     0.008
dtype: float64

Dropping columns: ['STDDEV_ITEM_PRICE']


## 4. Create Derived Features

In [42]:
# First, let's see what columns we have
print("Available columns:")
print(df.columns.tolist())

print("\n" + "="*60)
print("⚠️ SKIPPING DERIVED FEATURES TO PREVENT DATA LEAKAGE")
print("="*60)

# IMPORTANT: Derived features are DISABLED to prevent data leakage
# Features like review scores, delivery times, etc. can leak target information
# We'll only use basic numerical features that exist at order time

print("""
Derived features DISABLED:
- customer_engagement_score (uses review_score - LEAKS satisfaction)
- delivery_efficiency (uses actual delivery days - LEAKS delay)
- price_per_item (OK but not needed)
- freight_value_ratio (OK but not needed)

Reason: These features use information available AFTER the order outcome,
making them perfect predictors (data leakage). We need features that exist
at ORDER TIME only.
""")

print(f"\n✅ Using only pre-existing features to avoid leakage")
print(f"Current shape: {df.shape}")

Available columns:
['ORDER_ID', 'CUSTOMER_ORDER_COUNT', 'CUSTOMER_LIFETIME_VALUE', 'CUSTOMER_AVG_ORDER_VALUE', 'CUSTOMER_TENURE_DAYS', 'DAYS_SINCE_LAST_ORDER', 'ACTUAL_DELIVERY_DAYS', 'ESTIMATED_DELIVERY_DAYS', 'ORDER_YEAR', 'ORDER_QUARTER', 'ORDER_MONTH', 'ORDER_WEEK', 'ORDER_DAY', 'ORDER_DAY_OF_WEEK', 'ORDER_HOUR', 'ORDER_ON_WEEKEND', 'PRODUCT_WEIGHT_G', 'PRODUCT_LENGTH_CM', 'PRODUCT_HEIGHT_CM', 'PRODUCT_WIDTH_CM', 'PRODUCT_VOLUME_CM3', 'PRODUCT_PHOTOS_QTY', 'PRODUCT_ORDER_COUNT', 'PRODUCT_AVG_PRICE', 'SELLER_ORDER_COUNT', 'SELLER_AVG_ITEM_PRICE', 'IS_SAME_STATE', 'IS_SAME_CITY', 'TOTAL_UNIQUE_PRODUCTS', 'TOTAL_ITEMS', 'TOTAL_PRODUCT_VALUE', 'TOTAL_FREIGHT_VALUE', 'TOTAL_ORDER_VALUE', 'AVG_ITEM_PRICE', 'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE', 'MAX_INSTALLMENTS', 'AVG_INSTALLMENTS', 'PAYMENT_TYPES_COUNT', 'PAYMENT_CREDIT_CARD', 'PAYMENT_BOLETO', 'PAYMENT_VOUCHER', 'PAYMENT_DEBIT_CARD', 'REVIEW_SCORE', 'IS_POSITIVE_REVIEW', 'IS_NEGATIVE_REVIEW', 'FREIGHT_TO_PRODUCT_RATIO', 'AVG_VALUE_PER_IT

## 5. Prepare Datasets for 3 Prediction Tasks

In [43]:
# Define target columns and data leakage keywords
target_cols = ['is_delayed', 'is_canceled', 'is_satisfied']

# AGGRESSIVE keywords that indicate potential data leakage
# These features use information that's only available AFTER the order outcome
leakage_keywords = [
    'status', 'delay', 'cancel', 'delivered', 'review', 'score',
    'satisfaction', 'satisfied', 'target_', 'is_', 'actual',
    'rating', 'comment', 'feedback', 'diff', 'efficiency',
    'engagement', 'performance', 'quality'
]

# Drop non-feature columns
drop_cols = [
    'order_id', 'customer_id', 'seller_id', 'product_id',
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'shipping_limit_date',
    'review_creation_date', 'review_answer_timestamp',
    'review_comment_title', 'review_comment_message',
    'customer_unique_id', 'customer_zip_code_prefix',
    'customer_city', 'customer_state',
    'seller_zip_code_prefix', 'seller_city', 'seller_state',
    'product_category_name', 'product_category_name_english',
    'payment_type', 'order_status'
]

# Get feature columns (keep only numerical, exclude targets and leakage)
feature_cols = []
excluded_leakage = []

for col in df.columns:
    # Skip if not numerical
    if df[col].dtype not in [np.int64, np.float64]:
        continue
    
    # Skip if in drop list or target list
    if col in drop_cols or col in target_cols:
        continue
    
    # Check for data leakage keywords
    col_lower = col.lower()
    has_leakage = any(keyword in col_lower for keyword in leakage_keywords)
    
    if has_leakage:
        excluded_leakage.append(col)
    else:
        feature_cols.append(col)

print(f"Number of features: {len(feature_cols)}")
if excluded_leakage:
    print(f"\n⚠️ Excluded {len(excluded_leakage)} potential leakage features:")
    for col in excluded_leakage[:10]:
        print(f"  - {col}")
print(f"\n✅ Final features: {feature_cols[:10]}...")  # Show first 10

Number of features: 42

⚠️ Excluded 11 potential leakage features:
  - ACTUAL_DELIVERY_DAYS
  - IS_SAME_STATE
  - IS_SAME_CITY
  - REVIEW_SCORE
  - IS_POSITIVE_REVIEW
  - IS_NEGATIVE_REVIEW
  - IS_DELAYED
  - IS_CANCELED
  - IS_SATISFIED
  - TARGET_REVIEW_SCORE

✅ Final features: ['CUSTOMER_ORDER_COUNT', 'CUSTOMER_LIFETIME_VALUE', 'CUSTOMER_AVG_ORDER_VALUE', 'CUSTOMER_TENURE_DAYS', 'DAYS_SINCE_LAST_ORDER', 'ESTIMATED_DELIVERY_DAYS', 'ORDER_YEAR', 'ORDER_QUARTER', 'ORDER_MONTH', 'ORDER_WEEK']...


## 6. Split Data into Train/Validation/Test Sets

In [44]:
# Create data directory if not exists
data_dir = Path().absolute().parent / 'data'
data_dir.mkdir(exist_ok=True)

# First, check which target columns actually exist
print("Checking for target columns...")
expected_targets = ['is_delayed', 'is_canceled', 'is_satisfied']
existing_targets = [t for t in expected_targets if t in df.columns]

if not existing_targets:
    # Try to find any target-like columns
    target_candidates = [col for col in df.columns 
                        if any(keyword in col.lower() 
                              for keyword in ['delay', 'cancel', 'satisf', 'target_'])]
    print(f"\n⚠️ Expected targets not found!")
    print(f"Available target-like columns: {target_candidates}")
    existing_targets = target_candidates[:3] if target_candidates else []

print(f"\n✅ Using target columns: {existing_targets}")

if not existing_targets:
    print("\n❌ No target columns found! Cannot proceed with dataset preparation.")
    print("Please check your gold_obt_orders_ml_export table schema.")
else:
    # Prepare datasets for each task
    datasets = {}
    
    for target in existing_targets:
        print(f"\n{'='*60}")
        print(f"Preparing dataset for: {target}")
        print(f"{'='*60}")
        
        # Get features and target
        X = df[feature_cols].copy()
        y = df[target].copy()
        
        # Remove rows with missing target
        mask = y.notna()
        X = X[mask]
        y = y[mask]
        
        print(f"  Samples: {len(X):,}")
        print(f"  Class distribution: {y.value_counts().to_dict()}")
        
        # Split: 60% train, 20% validation, 20% test
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
        )
        
        print(f"  Train: {len(X_train):,}, Val: {len(X_val):,}, Test: {len(X_test):,}")
        
        datasets[target] = {
            'X_train': X_train, 'y_train': y_train,
            'X_val': X_val, 'y_val': y_val,
            'X_test': X_test, 'y_test': y_test
        }
    
    print(f"\n✅ Created {len(datasets)} datasets successfully!")

Checking for target columns...

⚠️ Expected targets not found!
Available target-like columns: ['IS_DELAYED', 'IS_CANCELED', 'IS_SATISFIED', 'TARGET_REVIEW_SCORE', 'TARGET_DELIVERY_DAYS']

✅ Using target columns: ['IS_DELAYED', 'IS_CANCELED', 'IS_SATISFIED']

Preparing dataset for: IS_DELAYED
  Samples: 50,000
  Class distribution: {0: 45997, 1: 4003}
  Train: 30,000, Val: 10,000, Test: 10,000

Preparing dataset for: IS_CANCELED
  Samples: 50,000
  Class distribution: {0: 50000}
  Train: 30,000, Val: 10,000, Test: 10,000

Preparing dataset for: IS_SATISFIED
  Samples: 50,000
  Class distribution: {1: 38658, 0: 11342}
  Train: 30,000, Val: 10,000, Test: 10,000

✅ Created 3 datasets successfully!


## 7. Feature Scaling

In [45]:
# Check if we have datasets to scale
if not datasets:
    print("❌ No datasets available for scaling. Run the previous cell first.")
else:
    # Get the actual target columns that were created
    actual_targets = list(datasets.keys())
    print(f"Scaling features for targets: {actual_targets}")
    
    # Fit scaler on the first target's training data
    scaler = StandardScaler()
    first_target = actual_targets[0]
    scaler.fit(datasets[first_target]['X_train'])
    
    # Apply scaling to all datasets
    for target in actual_targets:
        for split in ['X_train', 'X_val', 'X_test']:
            datasets[target][split] = pd.DataFrame(
                scaler.transform(datasets[target][split]),
                columns=datasets[target][split].columns,
                index=datasets[target][split].index
            )
    
    # Save scaler
    joblib.dump(scaler, data_dir / 'scaler.pkl')
    print("\n✅ Feature scaling completed and scaler saved")

Scaling features for targets: ['IS_DELAYED', 'IS_CANCELED', 'IS_SATISFIED']

✅ Feature scaling completed and scaler saved


## 8. Feature Selection

In [46]:
# Check if we have datasets for feature selection
if not datasets:
    print("❌ No datasets available for feature selection. Run the previous cells first.")
else:
    # Get the actual target columns
    actual_targets = list(datasets.keys())
    first_target = actual_targets[0]
    
    # Get number of features
    n_features = datasets[first_target]['X_train'].shape[1]
    k_best = min(30, n_features)  # Select top 30 or all if less than 30
    
    print(f"Selecting top {k_best} features from {n_features} total features")
    
    # Select top K features based on mutual information
    selector = SelectKBest(mutual_info_classif, k=k_best)
    
    # Fit selector on first target's training data
    selector.fit(
        datasets[first_target]['X_train'],
        datasets[first_target]['y_train']
    )
    
    # Get selected feature names
    selected_features = datasets[first_target]['X_train'].columns[selector.get_support()].tolist()
    print(f"\nSelected {len(selected_features)} features:")
    print(selected_features)
    
    # Apply feature selection to all datasets
    for target in actual_targets:
        for split in ['X_train', 'X_val', 'X_test']:
            datasets[target][split] = datasets[target][split][selected_features]
    
    # Save selector
    joblib.dump(selector, data_dir / 'feature_selector.pkl')
    print("\n✅ Feature selection completed and selector saved")

Selecting top 30 features from 42 total features

Selected 30 features:
['CUSTOMER_LIFETIME_VALUE', 'CUSTOMER_AVG_ORDER_VALUE', 'ESTIMATED_DELIVERY_DAYS', 'ORDER_YEAR', 'ORDER_QUARTER', 'ORDER_MONTH', 'ORDER_WEEK', 'ORDER_DAY_OF_WEEK', 'PRODUCT_LENGTH_CM', 'PRODUCT_HEIGHT_CM', 'PRODUCT_VOLUME_CM3', 'PRODUCT_PHOTOS_QTY', 'PRODUCT_ORDER_COUNT', 'PRODUCT_AVG_PRICE', 'SELLER_ORDER_COUNT', 'SELLER_AVG_ITEM_PRICE', 'TOTAL_PRODUCT_VALUE', 'TOTAL_FREIGHT_VALUE', 'TOTAL_ORDER_VALUE', 'AVG_ITEM_PRICE', 'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE', 'MAX_INSTALLMENTS', 'AVG_INSTALLMENTS', 'PAYMENT_CREDIT_CARD', 'PAYMENT_BOLETO', 'PAYMENT_VOUCHER', 'FREIGHT_TO_PRODUCT_RATIO', 'AVG_VALUE_PER_ITEM', 'TOTAL_CREDIT_EXTENDED']

✅ Feature selection completed and selector saved


## 9. Save Processed Datasets

In [47]:
# Check if we have datasets to save
if not datasets:
    print("❌ No datasets available to save. Run the previous cells first.")
else:
    # Save all datasets using actual target names
    print("Saving processed datasets...")
    
    for target in datasets.keys():
        task_name = target.replace('is_', '')
        
        for split_name, split_data in datasets[target].items():
            filename = f"{task_name}_{split_name}.parquet"
            
            # Convert Series to DataFrame if needed (for y_train, y_test, y_val)
            if isinstance(split_data, pd.Series):
                split_data = split_data.to_frame()
            
            split_data.to_parquet(data_dir / filename)
            print(f"  ✅ Saved: {filename}")
    
    print(f"\n✅ Feature engineering completed!")
    print(f"Processed data saved to: {data_dir}")
    print(f"Total files created: {len(list(data_dir.glob('*.parquet')))}")

Saving processed datasets...
  ✅ Saved: IS_DELAYED_X_train.parquet
  ✅ Saved: IS_DELAYED_y_train.parquet
  ✅ Saved: IS_DELAYED_X_val.parquet
  ✅ Saved: IS_DELAYED_y_val.parquet
  ✅ Saved: IS_DELAYED_X_test.parquet
  ✅ Saved: IS_DELAYED_y_test.parquet
  ✅ Saved: IS_CANCELED_X_train.parquet
  ✅ Saved: IS_CANCELED_y_train.parquet
  ✅ Saved: IS_CANCELED_X_val.parquet
  ✅ Saved: IS_CANCELED_y_val.parquet
  ✅ Saved: IS_CANCELED_X_test.parquet
  ✅ Saved: IS_CANCELED_y_test.parquet
  ✅ Saved: IS_SATISFIED_X_train.parquet
  ✅ Saved: IS_SATISFIED_y_train.parquet
  ✅ Saved: IS_SATISFIED_X_val.parquet
  ✅ Saved: IS_SATISFIED_y_val.parquet
  ✅ Saved: IS_SATISFIED_X_test.parquet
  ✅ Saved: IS_SATISFIED_y_test.parquet

✅ Feature engineering completed!
Processed data saved to: c:\Users\hakka\Documents\AWS_SNOWFLAKE_DBT_project\dbt_snowflake\ml_pipeline\data
Total files created: 18


## Summary

**Created Artifacts:**
- 18 parquet files (3 tasks × 6 files per task)
- `scaler.pkl` - StandardScaler fitted on training data
- `feature_selector.pkl` - SelectKBest with top 30 features

**Next Steps:**
1. Run `03_model_training.ipynb` to train ML models
2. Evaluate model performance
3. Deploy best models to production